MS peak align based on bioAssay peak alignment for excipient screening
see nb-dejejord-5094579-001

In [1]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import streamlit as st


In [2]:
#loading data 
data = pd.read_excel('./exampledata/Week2_Trimmed.xlsx')


pre-processing and qc steps
return parent peak retention time median, min RT, max RT, enter global parent peak RT (optional)
parent API peak cutoff


In [3]:
df_vial_counts = data.groupby(['Vial']).size().reset_index(name='peakCount')
maxarea = data.groupby(['Vial'])['Area'].max().reset_index()
areasum = data.groupby(['Vial'])['Area'].sum().reset_index(name='Sum of Area')
maxarea['Area Sum'] = areasum['Sum of Area']
maxarea['Area Ratio'] = maxarea['Area']/ maxarea['Area Sum']*100
RTmax= (data.sort_values(['Vial', 'Area'], ascending=[True, False])
             .drop_duplicates(['Vial']).reset_index(drop=True)
          )
maxarea = pd.merge(maxarea,RTmax,how = 'left', on = ['Vial', 'Area'])
maxarea = pd.merge(maxarea,df_vial_counts, how = 'left', on = 'Vial')
maxarea[['Plate','Vial_Well']] = maxarea['Vial'].str.split(':',expand=True)

In [4]:
print(data.head(10))

   Unnamed: 0 SampleName    Vial  Retention Time       Area     Height  \
0           9  2Week_40C   1:A,1          12.673   217673.0     4168.0   
1          10  2Week_40C   1:A,1           9.939    16884.0     3738.0   
2          13  2Week_40C   1:A,1           8.904  4405299.0  1029144.0   
3         135  2Week_40C  1:A,10           8.906  4743392.0  1098051.0   
4         136  2Week_40C  1:A,10           9.940    18490.0     4110.0   
5         118  2Week_40C  1:A,11           8.914  4287113.0   998123.0   
6         119  2Week_40C  1:A,11           9.949    16371.0     3642.0   
7         123  2Week_40C  1:A,12           8.918  5890075.0  1357347.0   
8         126  2Week_40C  1:A,12           9.946    22715.0     5008.0   
9          23  2Week_40C   1:A,2           9.939    24013.0     5314.0   

  Peak Type          Name  Injection  \
0   Unknown           NaN          1   
1     Found  API_Impurity          1   
2     Found           API          1   
3   Unknown           NaN

In [5]:

median_retention_time = maxarea['Retention Time'].median()
min_retention_time = maxarea['Retention Time'].min()
max_retention_time = maxarea['Retention Time'].max()
#print median, min and max retention time
print('Median retention time:', median_retention_time)
print('Minimum retention time:', min_retention_time)
print('Maximum retention time:', max_retention_time)

Median retention time: 8.903
Minimum retention time: 8.852
Maximum retention time: 9.987


calculate RRT values, count of vial peaks, sum of AreaRatio, Assign noise, Shift global RRT option to merge

In [6]:
#calculate relative rention time 'RRT' of all of the peaks
data['RRT'] = data.apply(lambda row: row['Retention Time'] / maxarea[maxarea['Vial'] == row['Vial']]['Retention Time'].values[0], axis=1)
data['RRT'] = data['RRT'].round(2)
#calculate Area ratio of all of the peaks
data['AreaRatio'] = data.apply(lambda row: row['Area']/ maxarea[maxarea['Vial'] == row['Vial']]['Area'].values[0], axis=1)
#count number of rows with each RRT value
df_RRT_counts = data.groupby(['RRT']).size().reset_index(name='peakCount')
#find the RRT value with the highest number of peaks
max_RRT = df_RRT_counts[df_RRT_counts['peakCount'] == df_RRT_counts['peakCount'].max()]['RRT'].values[0]
#return the sum of the area ratio values for each RRT value
df_RRT_area_ratio = data.groupby(['RRT'])['AreaRatio'].sum().reset_index(name='AreaRatio Sum')
df_RRT = pd.merge(df_RRT_counts, df_RRT_area_ratio, on='RRT')
print(df_RRT.head(10))

    RRT  peakCount  AreaRatio Sum
0  0.11          1       0.001787
1  0.32          4       0.037787
2  0.33          1       0.048430
3  0.34          1       0.005200
4  0.90          1       0.000000
5  1.00        145     144.000000
6  1.12        138       0.532429
7  1.17          7       0.622475
8  1.18          9       1.920207
9  1.41          1       0.002448


After you choose which peaks to shift, show what you are shifting in a table essentially

In [ ]:
# DataFrame to store selected rows
selected_rows_df = pd.DataFrame(columns=df.columns)

# Streamlit title
st.title("Dropdown Menu with Conditional RRT Merging")

# Function to find the row with the next least 'RRT' value for the same 'vial'
def find_next_least_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] > rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmin()]
        return next_row
    else:
        return None

# Function to find the row with the next highest 'RRT' value for the same 'vial'
def find_next_highest_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] < rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmax()]
        return next_row
    else:
        return None

# Display dropdown menus for each row
st.write("Select an action for each row:")
for index, row in df_RRT.iterrows():
    # Dropdown for selecting an action
    action = st.selectbox(
        f"Action for {row['Name']}:",
        options=['None', '< shift to left', '> shift to right'],
        key=index  # Use the row index as the key for unique identification
    )
    
    # If "< shift to left" is selected, add the row and the next least 'RRT' row to `selected_rows_df`
    if action == '< shift to left':
        selected_rows_df = selected_rows_df.append(row, ignore_index=True)
        next_row = find_next_least_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)
    
    # If "> shift to right" is selected, add the row and the next highest 'RRT' row to `selected_rows_df`
    elif action == '> shift to right':
        selected_rows_df = selected_rows_df.append(row, ignore_index=True)
        next_row = find_next_highest_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)

# Display the updated DataFrame
st.write("Selected Rows DataFrame:")
st.write(selected_rows_df)
